In [1]:
import asyncio

Let's consider the simplest possible coroutine for the sake of illustration

In [2]:
import time

async def mycoro(timeout):
    print("-> {} mycoro({})".format(time.strftime("%M-%S"), timeout))
    await asyncio.sleep(timeout)
    print("<- {} mycoro({})".format(time.strftime("%M-%S"), timeout))
    # return something easy to recognize
    return 1000 * timeout

# example 1

Running a series of coroutines in parallel - a la `gather` - can be done like this

In [3]:
from action import Action
from engine import Engine

In [4]:
a1, a2, a3 = Action(mycoro(1)), Action(mycoro(2)), Action(mycoro(1.5)),

What we're saying here is that we have three jobs, that have no relationships between them. 

So when we run them, we would start all 3 coroutines at once, and return once they are all done:

In [5]:
e = Engine(a1, a2, a3)
e.orchestrate()

-> 43-25 mycoro(1)
-> 43-25 mycoro(2)
-> 43-25 mycoro(1.5)
<- 43-26 mycoro(1)
<- 43-26 mycoro(1.5)
<- 43-27 mycoro(2)


True

# exemple 2 : add requirements (dependencies)

Now we can add *requirements* dependencies between actions, like in the following example. We take this chance to show that actions can be tagged with a label, which can turn out te be convenient somtimes.

In [6]:
b1, b2, b3 = (Action(mycoro(1), label="first"),
              Action(mycoro(2)), 
              Action(mycoro(1.5), label="last"))

b3.requires(b1)

Now `b3` needs `b1` to be finished before it can start. And so only the 2 first coroutines get started at the beginning, and only once b1 has finished does b3 start.

In [7]:
e2 = Engine(b1, b2, b3)
e2.orchestrate()

-> 43-27 mycoro(1)
-> 43-27 mycoro(2)
<- 43-28 mycoro(1)
-> 43-28 mycoro(1.5)
<- 43-29 mycoro(2)
<- 43-29 mycoro(1.5)


True

# Customizing the `Action` class

`Action` actually is a specializtion of `AbstractAction`, and the specification is that the `corun()` method should denote a coroutine itself, and that is what is triggered by `Engine` for running said action.

You can define your own `Action` class by specializing `action.AbstractAction`.

# Inspect results

Details for each `Action` can be retrieved like this

In [8]:
print(b1.is_done())

True


In [9]:
print(b3.result())

1500.0


In [10]:
for action in e2.actions:
    print(action)

<Action `last' finished -> 1500.0 - [requires [first]]>
<Action `first' finished -> 1000 - [allows [last]]>
<Action `NO_LABEL' finished -> 2000>


# TODO

This totally is only a seed at this point, like day d+1 

## termination
* orchestrate should accept a timeout argument
* some actions are not designed to end. like, a monitoring tool that keeps on looking for a message queue and react.. It feels like if we provided a means to declare an `Action` as being such an never-ending loop, `orchestrate` would then be in a position to handle that gracefully.
* provide a means to shutdown actions once the engine has run out. Typically we would have several actions using the same ssh connection, and these need to be closed at some point. Something like `Engine.shutdown` or similar.

  Would it work to just send e.g. `coshutdown()` on all actions ? this needs a little more thinking though; that could mean trying to shutdown an ssh connection from several points at the same time ...

## deal with exceptions
* for now we kind of assume that `corun()` does not trigger an exception. This needs to be robustified. 

## monitoring 
* come up with some basic (curses ?) monitor to show what's going on; what I have in mind is something like rhubarbe load where all actions would be displayed, one line each, and their status could be shown so that one can get a sense of what is going on
* one way to look at this is to have the main Engine class send itself a `tick()` method, and then specialize `Engine` as `EngineCurses` that would actually do things on such events.
* ***or*** this gets delegated on a `message_queue` object. **Review the rhubarbe code on this aspect**.
